In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
from scipy.stats import pearsonr

from sklearn.preprocessing import StandardScaler
from sklearn import metrics 
from sklearn.model_selection import KFold, GridSearchCV

import time
import random

from utils import data_handler, plotter

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")


#handle warnings
import warnings
warnings.filterwarnings("ignore")

np.random.seed(3)

In [ ]:
plt.rc('font', family='serif')

# set font size
SMALL_SIZE = 22
MEDIUM_SIZE=24
BIGGER_SIZE = 26

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

Metrics: 
r2,r,MSE - coefﬁcient of determination (R2), Pearson coefﬁcient (r), and mean square error (MSE) 

In [ ]:
def test(y_pred,y_true,end_ptr):
    '''Evaluate the performance of IAM through 3 metrics; 
       And find the best predicted condition for the next IAM trial
       
       Inputs:
           X_test      - for testing
           y_true      - ground true for X_test
           end_ptr     - range of [0,end_ptr] in y_pred and y_true are values observable to IAM
           
       Uses:(global variables)    
           pred_result - of shape (total_samp,), storing all the results predicted by IAM
           true_results- of shape (total_samp,), the same order as of pred_results, storing all the ground truth
           
       Outputs:
            
    
    '''
    #All sample metrics    
    r2 = metrics.r2_score(y_true,y_pred)   
    mse = metrics.mean_squared_error(y_true,y_pred)
    [pear,p_value] = pearsonr(y_true,y_pred)
    
    #metrics based on observable samples
    y_true_s = y_true[0:(end_ptr+1)]
    y_pred_s = y_pred[0:(end_ptr+1)]
    r2_s = metrics.r2_score(y_true_s,y_pred_s)   
    mse_s = metrics.mean_squared_error(y_true_s,y_pred_s)
    [pear_s,p_value_s] = pearsonr(y_true_s,y_pred_s)
    return [r2,pear,p_value,mse,r2_s,pear_s,p_value_s,mse_s] #, best_pos_ind, best_prob 


# Data Handling

In [ ]:
X_df,Y_df = data_handler.load_XY(1)
X = X_df.as_matrix()
Y = Y_df.as_matrix()/100

# Set up & construct initial dataset

In [ ]:
# cross validation settup
inner_nsplits = 10
init_train_size = 20
totalSamp = X.shape[0]


Y_global_max = np.max(Y)
all_ind = np.random.permutation(list(range(0,totalSamp)))
all_ind_wo_max = list(range(0,totalSamp))
all_ind_wo_max.remove(0)

# PAM guided sythesis

In [ ]:
# setup
Nc = 0
save_csv= True
verbose=True
title='cqd_PAM_'
batch=1
to_break = False # whether to break when the best condition is found

## start IAM guided synthesis...
init_time = time.time()

#construct initial training set
results_mat = np.zeros(((totalSamp-init_train_size),12))


train_ind = random.sample(all_ind_wo_max, init_train_size)
test_ind = [x for x in all_ind if x not in train_ind]

# set up result storage to compute eval metrics, in the order of IAM
#  ignore the initial training set, as it is not determined by IAM
pred_results = np.zeros(totalSamp-init_train_size)
true_results = np.zeros(totalSamp-init_train_size)


# setup the hyperparameter range for tuning
tuned_parameters = dict(learning_rate=[0.01],
                        n_estimators=[300,500,700], #100,,300,400,500
                        colsample_bylevel = [0.5,0.7,0.9],
                      gamma=[0,0.2], #0,0.1,0.2,0.3,0.4
                      max_depth =[3,7,11], # [3,7,11]]
                      reg_lambda = [0.1,1,10], #[0.1,1,10]
                     # reg_alpha = [1],
                       subsample=[0.4,0.7,1])

j=0
loop_count = 0
mean_y_only_init = np.mean(Y[train_ind])
std_y_only_init = np.std(Y[train_ind])


while(j<totalSamp-init_train_size):        
    inner_cv = KFold(n_splits=inner_nsplits,shuffle=True, random_state=j) 
    X_train = X[train_ind]
    Y_train = Y[train_ind]
    X_test = X[test_ind]
    Y_test = Y[test_ind]

    last_max = np.max(Y_train)

    # GradientBoost
    reg = xgb.XGBRegressor(objective="reg:linear",min_child_weight=1,**{'tree_method':'exact'},
                             silent=True,n_jobs=4,random_state=3,seed=3);

    gb_clf = GridSearchCV(reg,tuned_parameters, cv=inner_cv,scoring='r2',verbose=0,n_jobs=4)
    gb_clf.fit(X_train, Y_train)
    y_pred = gb_clf.predict(X_test)


    # choose the condition with best predicted yield
    best_pos_ind = np.argsort(-y_pred)[:batch]       
    best_prob = y_pred[best_pos_ind]
    next_ind = np.array(test_ind)[best_pos_ind]        

    # update results storage
    train_size = len(Y_train)
    temp = list(range(0,len(y_pred)))
    ind_notbest = [x for x in temp if x not in best_pos_ind]

    start_ptr = j
    end_ptr = np.min([start_ptr + batch, totalSamp-init_train_size]) 
    pred_results[start_ptr:end_ptr] = best_prob
    pred_results[end_ptr:totalSamp-init_train_size]= y_pred[ind_notbest]
    true_results[start_ptr:end_ptr] = Y_test[best_pos_ind]
    true_results[end_ptr:totalSamp-init_train_size] = Y_test[ind_notbest]

    pred_metrics = test(pred_results,true_results,end_ptr-1)    

    # calculate results
    next_best_true_ind = next_ind[np.argmax(Y_test[best_pos_ind])]
    next_best_y_true = np.max(Y_test[best_pos_ind])
    result_list = [train_size,next_best_true_ind,next_best_y_true,best_prob[0],] + pred_metrics   
    results_mat[loop_count,:] = np.array(result_list)

    loop_count = loop_count +1
    j = j + batch

    if(verbose):
        print(loop_count,'->',j,', best_next_ind=',next_best_true_ind, ' best_Y_true=',"{0:.6f}".format(next_best_y_true),' train_max=',"{0:.6f}".format(last_max),' r2=',pred_metrics[0])

    train_ind = [*train_ind , *next_ind]   
    test_ind = [x for x in test_ind if x not in next_ind]

    ## critical point
    if(next_best_y_true==Y_global_max):
        Nc = j+init_train_size
        print('***Nc=', str(Nc))
        if(to_break):
            break


results = pd.DataFrame(data=results_mat[0:j,:],columns=['sample_size','pred_ind','best_pred_result','y_true','r2','pearson','p_value','mse','r2_s','pearson_s','p_value_s','mse_s'])

if(save_csv):
    data_handler.save_csv(results,title=title+'_Nc_'+str(Nc))


run_time = (time.time() - init_time)/60

print('end at loop ',j)
print('critical point is when size of training set = ',Nc)
print('total time = ',run_time,' mins')

## Plotting

In [ ]:
n_samp = results['sample_size']
pred_results = results['best_pred_result'] 
true_results = results['y_true'] 

smoothing

In [ ]:
w = 3 # window size

seq_len = len(pred_results)
pad = int((w-1)/2)
pred_norm = np.convolve(pred_results, np.ones(w)*1/w , mode='valid')#medfilt(pred_results,w)[pad:seq_len-pad]##max_pooling(pred_results, w=9 , s=1)#
#pred_norm = medfilt(pred_norm,3)#[1:len(pred_norm)-1]
true_norm = np.convolve(true_results, np.ones(w)*1/w , mode='valid')#max_pooling(true_results, w=w , s=1)#np.convolve(true_results, np.array([0.2, 0.2, 0.2, 0.2, 0.2]) , mode='valid')#
n_samp_new = n_samp[pad:seq_len-pad]

# save csv
to_save_mat = np.zeros((len(n_samp_new), 3))
to_save_mat[:,0]  = n_samp_new
to_save_mat[:,1] = pred_norm
to_save_mat[:,2] = true_norm

to_save_df = pd.DataFrame(to_save_mat, columns=['n_sample','pred','true'])
save_path = data_handler.save_csv(to_save_df, title=title+'_smoothed')

plotting

In [ ]:
fig = plt.figure(figsize = (12,12/1.618),dpi=100)
#y = Y[train_ind]
plt.plot(n_samp_new,pred_norm,lw=2, label='Best Predicted', linestyle='-',color='b')
plt.plot(n_samp_new,true_norm,lw=2, label='Best True', linestyle='--',color='r')
#labels = list(range(30,np.max(n_samp),50)) + [n_samp]
plt.axvline(x=Nc,linestyle='--',color='k')
#plt.xticks(labels)  

plt.grid(False)

plt.legend(loc='lower left')
plt.xlabel('Number of Explored Conditions')
plt.ylabel('PLQY')
plt.show()

save_path = plotter.save_fig(fig, title+'_smoothed')

In [ ]:
end_time = time.time()
print('total time = ',(end_time - init_time)/60,' mins')